# Data Generation from Scopus database

I created five tables for each subfield in AI. Each subfield table contains the articles from both mapped conferences:

- AI (*scb_b_202401_venue_items_ai*)
  - AAAI
  - IJCAI
- CV (*scb_b_202401_venue_items_cv*)
  - CVPR
  - ICCV
- ML (*scb_b_202401_venue_items_ml*)
  - ICML
  - ICLR
- NLP (*scb_b_202401_venue_items_nlp*)
  - ACL
  - EMNLP
- WIR (*scb_b_202401_venue_items_wir*)
  - WSDM
  - SIGIR

**Calculate outgoing citations from *industry-funded* papers to industry-funded papers**

In [ ]:
-- cited column contains id's of articles that are not included in scopus.
-- thus, we can't declare their fundig type --> we can't use them in the analysis.
CREATE table unigjpwahle.unique_refs AS
SELECT DISTINCT r.item_id_citing, r.item_id_cited
FROM scp_b_202401.refs r
inner JOIN scp_b_202401.items i ON i.item_id = r.item_id_cited;

WITH all_articles AS (
    SELECT DISTINCT item_id, pubyear
    FROM unigjpwahle.scb_b_202401_venue_items_ai -- this table contains all articles from 2018-2023 for the two selected AI venues
    WHERE pubyear IN (2018, 2019, 2020, 2021, 2022, 2023)
),
-- declare funding type for each article in scb_b_202401_venue_items_ai
article_funding_type AS (
    SELECT 
        aa.item_id, aa.pubyear,
        CASE 
            WHEN fg.item_id IS NULL THEN 'non-funded'
            WHEN EXISTS (
                SELECT 1
                FROM scp_b_202401.funding_agencies_grants fg2
                INNER JOIN unigjpwahle.extracted_funding_agencies efa ON fg2.funding_agency = efa.funding_agencies
                WHERE fg2.item_id = aa.item_id
            ) THEN 'industry'
            ELSE 'non-industry'
        END AS funding_type
    FROM all_articles aa
    LEFT JOIN scp_b_202401.funding_agencies_grants fg ON aa.item_id = fg.item_id
),
-- get all items that are industry funded in scb_b_202401_venue_items_ai
industry_funded as (
select distinct item_id, pubyear
from article_funding_type
where funding_type = 'industry'
),
-- declare funding type for each article in scopus database (items table)
items_funding_type as (
    SELECT 
        aa.item_id, aa.pubyear,
        CASE 
            WHEN fg.item_id IS NULL THEN 'non-funded'
            WHEN EXISTS (
                SELECT 1
                FROM scp_b_202401.funding_agencies_grants fg2
                INNER JOIN unigjpwahle.extracted_funding_agencies efa ON fg2.funding_agency = efa.funding_agencies
                WHERE fg2.item_id = aa.item_id
            ) THEN 'industry'
            ELSE 'non-industry'
        END AS funding_type
    FROM scp_b_202401.items
    LEFT JOIN scp_b_202401.funding_agencies_grants fg ON aa.item_id = fg.item_id
),
-- get all items that are industry funded from scopus database
items as (
select distinct item_id, pubyear
from items_funding_type
where funding_type = 'industry' -- change to 'industry' to get industry funded items
)
select
	inf.pubyear,
	COUNT(DISTINCT r.*) as citation_count
FROM unigjpwahle.unique_refs r
inner join industry_funded inf ON r.item_id_citing = inf.item_id -- get references from industry funded articles in scb_b_202401_venue_items_ai
inner join items i ON r.item_id_cited = i.item_id -- get all referenced articles that are industry funded in scopus database
GROUP BY inf.pubyear
ORDER BY inf.pubyear;

**Calculate outgoing citations from *non-funded* papers to industry-funded papers**

In [ ]:
WITH all_articles AS (
    SELECT DISTINCT item_id, pubyear
    FROM unigjpwahle.scb_b_202401_venue_items_ai
    WHERE pubyear IN (2018, 2019, 2020, 2021, 2022, 2023)
),
-- declare funding type for each article in scb_b_202401_venue_items_ai
article_funding_type AS (
    SELECT 
        aa.item_id, aa.pubyear,
        CASE 
            WHEN fg.item_id IS NULL THEN 'non-funded'
            WHEN EXISTS (
                SELECT 1
                FROM scp_b_202401.funding_agencies_grants fg2
                INNER JOIN unigjpwahle.extracted_funding_agencies efa ON fg2.funding_agency = efa.funding_agencies
                WHERE fg2.item_id = aa.item_id
            ) THEN 'industry'
            ELSE 'non-industry'
        END AS funding_type
    FROM all_articles aa
    LEFT JOIN scp_b_202401.funding_agencies_grants fg ON aa.item_id = fg.item_id
),
-- get all items that are non-funded in scb_b_202401_venue_items_ai
non_funded as (
select distinct item_id, pubyear
from article_funding_type
where funding_type = 'non-funded'
),
-- declare funding type for each article in scopus database (items table)
items_funding_type as (
    SELECT 
        aa.item_id, aa.pubyear,
        CASE 
            WHEN fg.item_id IS NULL THEN 'non-funded'
            WHEN EXISTS (
                SELECT 1
                FROM scp_b_202401.funding_agencies_grants fg2
                INNER JOIN unigjpwahle.extracted_funding_agencies efa ON fg2.funding_agency = efa.funding_agencies
                WHERE fg2.item_id = aa.item_id
            ) THEN 'industry'
            ELSE 'non-industry'
        END AS funding_type
    FROM scp_b_202401.items aa
    LEFT JOIN scp_b_202401.funding_agencies_grants fg ON aa.item_id = fg.item_id
),
-- get all items that are industry funded from scopus database
-- change to 'industry' to get industry funded items
-- change to 'non-industry' to get non-industry items
items as (
select distinct item_id, pubyear
from items_funding_type
where funding_type = 'industry'
)
select
	inf.pubyear,
	COUNT(DISTINCT r.*) as citation_count
FROM unigjpwahle.unique_refs r
inner join non_funded inf ON r.item_id_citing = inf.item_id -- get references from non-funded articles in scb_b_202401_venue_items_ai
inner join items i ON r.item_id_cited = i.item_id -- get all referenced articles that are industry funded in scopus database
GROUP BY inf.pubyear
ORDER BY inf.pubyear;

# Data Processing

**Avg. Outgoing Citations per Industry-Funded Paper, 2018-2023**

In [ ]:
excel_path = "{PATH}/master_thesis_data_analysis.xlsx"
data <-  read_excel(excel_path, sheet = "total_industry_to_X_cit_year", na = c("", " "))
data_funding <-  read_excel(excel_path, sheet = "funded_papers_over_time_field", na = c("", " "))

# Calculate the total number of citations per year
data_overall <- data %>% 
  select(-Group, -total_non_funded_to_industry_citations, -total_non_funded_to_non_funded_citations, -total_non_funded_to_non_industry_funded_citations, total_industry_funded_papers, total_non_funded_papers, total_non_industry_funded_papers) %>%
  group_by(Year) %>% 
  reframe(total_industry_to_industry_citations = sum(total_industry_to_industry_citations, na.rm = TRUE),
          total_industry_to_non_industry_funded_citations = sum(total_industry_to_non_industry_funded_citations, na.rm = TRUE),
          total_industry_to_non_funded_citations = sum(total_industry_to_non_funded_citations, na.rm = TRUE),
          total_industry_funded_papers = sum(total_industry_funded_papers, na.rm = TRUE),
          total_non_funded_papers = sum(total_non_funded_papers, na.rm = TRUE),
          total_non_industry_funded_papers = sum(total_non_industry_funded_papers, na.rm = TRUE))

# Calculate the average number of citations per industry-funded paper
data_avg <- data_overall %>% 
  group_by(Year) %>%
  reframe(total_industry_to_non_funded_citations_avg = (total_industry_to_non_funded_citations / total_industry_funded_papers),
         total_industry_to_non_industry_funded_citations_avg = (total_industry_to_non_industry_funded_citations / total_industry_funded_papers),
         total_industry_to_industry_citations_avg = (total_industry_to_industry_citations / total_industry_funded_papers))

# Reshape the data to a longer format
data_long_overall <- data_avg %>%
  select(Year, total_industry_to_non_funded_citations_avg, total_industry_to_non_industry_funded_citations_avg, total_industry_to_industry_citations_avg) %>%
  pivot_longer(cols = c(total_industry_to_non_funded_citations_avg, total_industry_to_non_industry_funded_citations_avg, total_industry_to_industry_citations_avg),
               names_to = "Citation_Type",
                values_to = "Count") %>%
  mutate(Citation_Type = factor(Citation_Type, 
                                levels = c("total_industry_to_non_funded_citations_avg",
                                           "total_industry_to_non_industry_funded_citations_avg",
                                           "total_industry_to_industry_citations_avg"),
                                labels = c("Industry-to-Non-funded", "Industry-to-Non-industry-funded",
                                           "Industry-to-industry")))

data_long_overall_industry <- data_long_overall %>% filter(Citation_Type == "Industry-to-industry")
data_long_overall_non_industry <- data_long_overall %>% filter(Citation_Type == "Industry-to-Non-industry-funded")
data_long_overall_non_funded <- data_long_overall %>% filter(Citation_Type == "Industry-to-Non-funded")

# Plotting
subtitle_text <- "Avg. Outgoing Citations per Industry-Funded Paper, 2018-2023"

colors <- c("Industry" = "#1B068DFF", 
            "Non-industry" = "#AE347BFF",
            "Non-funded" = "#FCB216FF")

p <- ggplot() +
  geom_line(data=data_long_overall_industry, aes(x = Year, y = Count, color="Industry"), lwd = 2) +
    geom_line(data=data_long_overall_non_industry, aes(x = Year, y = Count, color="Non-industry"), lwd = 2) +
      geom_line(data=data_long_overall_non_funded, aes(x = Year, y = Count, color="Non-funded"), lwd = 2) +
  labs(x = "Year",
       y = "Normalized Outgoing Citations") +
  labs(subtitle = subtitle_text) +
  theme_minimal() +
  scale_color_manual(values = colors, breaks = c("Industry", "Non-industry", "Non-funded"), labels = c("To-Industry-Funded", "To-Non-Industry-Funded", "To-Non-Funded")) +
    bbc_style() +
theme(
    legend.position = "top",
    legend.justification='left',
    plot.subtitle = element_text(size = 19.4),
     axis.text.x = ggplot2::element_text(margin=ggplot2::margin(12, b = 0)),
         axis.ticks = element_line(size = 0.5, color = "black"),
    axis.ticks.length = unit(0.25, "cm"),
    axis.ticks.y = element_blank(),
    axis.line.x = element_line(colour = "#333333", linewidth = 1, linetype = "solid"))

ggsave("out_cit_industry.pdf", p, height = 6.67, width = 6.67)

print(p)

**Citation Bias Ratio of Industry-Funded Papers, 2018-2023**

In [ ]:
excel_path = "{PATH}/master_thesis_data_analysis.xlsx"
data <-  read_excel(excel_path, sheet = "total_industry_to_X_cit_year", na = c("", " "))
data_funding <-  read_excel(excel_path, sheet = "funded_papers_over_time_field", na = c("", " "))

# Calculate the total number of citations per year
data_overall <- data %>% 
  select(-Group, -total_non_funded_to_industry_citations, -total_non_funded_to_non_funded_citations, -total_non_funded_to_non_industry_funded_citations, total_industry_funded_papers, total_non_funded_papers, total_non_industry_funded_papers) %>%
  group_by(Year) %>% 
  reframe(total_industry_to_industry_citations = sum(total_industry_to_industry_citations, na.rm = TRUE),
          total_industry_to_non_industry_funded_citations = sum(total_industry_to_non_industry_funded_citations, na.rm = TRUE),
          total_industry_to_non_funded_citations = sum(total_industry_to_non_funded_citations, na.rm = TRUE),
          total_industry_funded_papers = sum(total_industry_funded_papers, na.rm = TRUE),
          total_non_funded_papers = sum(total_non_funded_papers, na.rm = TRUE),
          total_non_industry_funded_papers = sum(total_non_industry_funded_papers, na.rm = TRUE))

total_papers <- data_overall %>% 
  group_by(Year) %>%
  reframe(total_papers = sum(total_industry_funded_papers, total_non_funded_papers, total_non_industry_funded_papers, na.rm = TRUE))

total_citations <- data_overall %>% 
  group_by(Year) %>%
  reframe(total_citations = sum(total_industry_to_non_funded_citations, total_industry_to_non_industry_funded_citations, total_industry_to_industry_citations, na.rm = TRUE))

data_overall <- data_overall %>% left_join(total_papers, by = "Year") %>% left_join(total_citations, by = "Year")

# Proportion
data_overall <- data_overall %>% 
  group_by(Year) %>%
  mutate(p_industry = (total_industry_funded_papers / total_papers),
         p_non_industry_funded = (total_non_industry_funded_papers / total_papers),
         p_non_funded = (total_non_funded_papers / total_papers))

#  Expected Citations Based on Availability
data_overall <- data_overall %>% 
  group_by(Year) %>%
  mutate(e_c_industry = total_citations * p_industry,
         e_c_non_industry = total_citations * p_non_industry_funded,
         e_c_non_funded = total_citations * p_non_funded)

# Normalize Citation Counts
data_overall <- data_overall %>% 
  group_by(Year) %>%
  mutate(c_rate_industry = total_industry_to_industry_citations / total_industry_funded_papers,
         c_rate_non_industry = total_industry_to_non_industry_funded_citations / total_non_industry_funded_papers,
         c_rate_non_funded = total_industry_to_non_funded_citations / total_non_funded_papers)

data_overall <- data_overall %>% 
  group_by(Year) %>%
  mutate(e_rate_industry = e_c_industry / total_industry_funded_papers,
         e_rate_non_industry = e_c_non_industry / total_non_industry_funded_papers,
         e_rate_non_funded = e_c_non_funded / total_non_funded_papers)

# Citation Bias Ratio
data_overall <- data_overall %>% 
  group_by(Year) %>%
  reframe(cbr_industry = c_rate_industry / e_rate_industry,
         cbr_non_industry = c_rate_non_industry / e_rate_non_industry,
         cbr_non_funded = c_rate_non_funded / e_rate_non_funded)

# Plot
subtitle_text <- "CBR of Industry-Funded Papers, 2018-2023"

colors <- c("Industry" = "#1B068DFF", 
            "Non-industry" = "#B52F8CFF",
            "Non-funded" = "#F58C46FF")

p <- ggplot() +
  geom_hline(yintercept = 1, color = "#333333", size = 1, linetype="dotted") +
  geom_line(data=data_overall, aes(x = Year, y = cbr_industry, color="Industry"), lwd = 2) +
    geom_line(data=data_overall, aes(x = Year, y = cbr_non_industry, color="Non-industry"), lwd = 2) +
      geom_line(data=data_overall, aes(x = Year, y = cbr_non_funded, color="Non-funded"), lwd = 2) +
  labs(x = "Year",
       y = "Normalized Outgoing Citations") +
  theme_minimal() +
  labs(subtitle = subtitle_text) +
  scale_color_manual(values = colors, breaks = c("Industry", "Non-industry", "Non-funded"), labels = c("Industry", "Non-Industry", "Non-Funded")) +
  bbc_style() +
  theme(
    legend.position = "top",
    legend.justification='left',
    plot.subtitle = element_text(size = 19.4),
     axis.text.x = ggplot2::element_text(margin=ggplot2::margin(12, b = 0)),
         axis.ticks = element_line(size = 0.5, color = "black"),
    axis.ticks.length = unit(0.25, "cm"),
    axis.ticks.y = element_blank(),
    axis.line.x = element_line(colour = "#333333", linewidth = 1, linetype = "solid"))

ggsave("out_cit_industry_cbr.pdf", p, height = 6.67, width = 6.67)

print(p)